<a href="https://colab.research.google.com/github/Dkepffl/2022-2-ESAA/blob/main/Assignment/Assignment221118_recommedation3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **CHAPTER 09 추천 시스템**
___


## **08 파이썬 추천 시스템 패키지-Surprise**
___


In [1]:
# Surprise 패키지 설치 코드(colab)
!pip install scikit-surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### | **Surprise 패키지 소개**
- 파이썬 기반의 추천 시스템 구축 전용 패키지 중 하나인 Surprise를 소개한다.
- Surprise는 파이썬 기반에서 사이킷런과 유사한 API와 프레임워크를 제공한다.




### | **Surprise를 이용한 추천 시스템 구축**
- Surprise 패키지의 사용법을 알아보기 위해, 추천 데이터를 학습용과 테스트용 데이터셋으로 분리한 뒤, SVD 행렬 분해를 통한 잠재 요인 협업 필터링을 수행해보자.
- 우선, 필요한 모듈을 아래와 같이 호출한다.

In [2]:
import pandas as pd
from surprise import SVD
from surprise import Dataset 
from surprise import accuracy 
from surprise.model_selection import train_test_split

- 이제, 데이터셋을 로딩해보자.
- Surprise에서 데이터 로딩은 Dataset 클래스를 이용해야 한다. 
- Surprise는 Movie Lens 데이터셋의 사용자-영화 평점 데이터처럼 `userId`(사용자 ID), `movieId`(영화 ID), `rating`(평점)과 같은 주요 데이터가 Row 레벨 형태로 되어 있는 형태의 데이터만 처리한다.
- Surprise의 API를 이용하여 무비렌즈(MovieLens) 사이트에서 제공하는 과거 버전의 데이터셋을 불러올 수 있다.
  + `ml-100k` : 10만 개의 평점 데이터
  + `ml-1m` : 100만 개의 평점 데이터
- Surprise Dataset 클래스의 `load_bulletin()`은 데이터셋을 사이트에서 다운받아 로컬 디렉터리에 저장한 뒤, 데이터를 로딩한다.
- 데이터셋을 로딩하고, 학습/테스트 데이터셋으로 분리해보자.

In [ ]:
# load_builtin 사용
data = Dataset.load_builtin('ml-100k') 
trainset, testset = train_test_split(data, test_size=.25, random_state=0) 

In [4]:
ratings = pd.read_csv('/content/drive/MyDrive/2022-2 ESAA/Data/archive/ratings.csv')
ratings.to_csv('ratings_surprise.csv', index=False, header=False)

In [9]:
from surprise import Reader
from surprise.model_selection import train_test_split

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
data = Dataset.load_from_file('/content/ratings_surprise.csv',reader=reader)

In [10]:
trainset, testset = train_test_split(data, test_size = 0.25, random_state = 0)

- `load_builtin` API 사용 시 오류가 발생하여, 데이터셋을 다운 받아 사용하였다.
- 이제 SVD로 잠재요인 협업 필터링을 수행해보자.
- 적용하는 데이터셋은 앞에서 분리한 학습데이터셋이다.

In [12]:
algo = SVD() # 알고리즘 객체 생성
algo.fit(trainset) # 추천 알고리즘 학습

- 학습된 추천 알고리즘을 기반으로 테스트 데이터셋에 대해 추천을 수행하자.
- Surprise에서 추천을 예측하는 메서드는 test()와 predict(), 2개이다.
 - `test()` : 사용자-아이템 평점 데이터셋 전체에 대해 추천을 예측하는 메서드
 - `predict()` 개별 사용자와 영화에 대한 추천 평점 반환
- 예제를 통해 차이를 확인해보자.

In [13]:
predictions = algo.test( testset )
print('prediction type :',type(predictions), ' size:',len(predictions))
print('prediction 결과의 최초 5개 추출')
predictions[:5]

prediction type : <class 'list'>  size: 26335
prediction 결과의 최초 5개 추출


[Prediction(uid='576', iid='85774', r_ui=5.0, est=3.721264743127036, details={'was_impossible': False}),
 Prediction(uid='575', iid='45512', r_ui=2.5, est=3.0630913838607325, details={'was_impossible': False}),
 Prediction(uid='303', iid='1188', r_ui=3.0, est=4.024780624096519, details={'was_impossible': False}),
 Prediction(uid='363', iid='2898', r_ui=4.0, est=3.5498712837460626, details={'was_impossible': False}),
 Prediction(uid='501', iid='5293', r_ui=3.0, est=2.9392213042264332, details={'was_impossible': False})]

- SVD 알고리즘 객체의 `test(dataset)` 메서드의 호출 결과는 파이썬의 리스트 형태이며, 크기는 입력한 데이터셋과 크기가 같다.
- 호출 결과로 반환된 리스트 객체는 Predicton 객체를 가지고 있는데, 이 객체는 Surprise 패키지에서 제공하는 데이터 타입이며, 개별 사용자 아이디, 영화 아이디, 실제 평점 등에 기반한 추천 예측 평점 데이터를 튜플 형태로 가지고 있다.
- Prediction 객체에서 `uid`, `iid`, `est` 속성을 추출해보자.

In [14]:
[(pred.uid, pred.iid, pred.est) for pred in predictions[:3]]

[('576', '85774', 3.721264743127036),
 ('575', '45512', 3.0630913838607325),
 ('303', '1188', 4.024780624096519)]

- 이번에는 또다른 추천 예측 메서드인 `predict()`을 이용해 추천 예측을 해보자.
- `predict()`은 개별 사용자의 아이템에 대한 추천 평점을 예측해준다.
- 개별 사용자 아이디, 아이템 아이디를 입력하면 추천 예측 평점을 포함한 정보를 반환한다.

In [15]:
# 사용자 아이디, 아이템 아이디는 문자열로 입력해야 함
uid = str(196)
iid = str(302)
pred = algo.predict(uid, iid)
print(pred)

user: 196        item: 302        r_ui = None   est = 3.21   {'was_impossible': False}


- 위와 같이 `predict()`은 추천 예측 평점을 `est`로 반환한다.
- `test()` 메서드는 입력 데이터셋의 모든 사용자와 아이템 아이디에 대해 `predict()`을 반복적으로 수행한 결과라고 생각할 수 있다.
- 테스트 데이터셋을 이용해 추천 예측 평점과 실제 평점과의 차이를 평가해보자.

In [16]:
accuracy.rmse(predictions)

RMSE: 0.8613


0.8612526530775636

### | **Surprise 주요 모듈 소개**

##### **Dataset**
- Surprise는 `user_id`, `item_id`, `rating` 데이터가 로우 레벨로 된 데이터셋만 적용할 수 있다.
- 무비렌즈 아카이브 서버에서 자동으로 내려받는 데이터뿐만 아니라, 일반 데이터 파일이나 판다스 DataFrame도 로딩할 수 있으나, 컬럼의 순서를 지켜야 한다.
  - `Dataset.load_builtin(name='ml-100k')` : 무비렌즈 아카이브 FTP 서버에서 무비렌즈 데이터를 다운받는다.
  - `Dataset.load_from_file(flie_path, reader)` : OS 파일에서 데이터를 로딩한다.
  - `Dataset.load_from_df(df, reader)` : 판다스의 데이터프레임을 데이터로 로딩한다. 컬럼 순서에 유의해야 한다. 

##### **OS 파일 데이터를 Surprise 데이터셋으로 로딩**
- 이전 절에서 사용한 `ratings.csv`와 `movies.csv`를 `Dataset.load_from_file` API를 이용하여 로딩해보자.
- 로딩되는 데이터 파일에 컬럼명을 가지는 헤더 문자열이 있으면 안 된다는 것에 유의한다.

In [17]:
import pandas as pd

ratings = pd.read_csv('/content/drive/MyDrive/2022-2 ESAA/Data/archive/ratings.csv')

# ratings_noh.csv 파일로 언로드 시 index 와 header를 모두 제거한 새로운 파일 생성 
ratings.to_csv('ratings_noh.csv', index=False, header=False)

- 헤더를 삭제한 `ratings_noh.csv` 파일을 Dataset 모듈의 `load_from_file()`을 이용하여 DataSet으로 로드해보자.
- `line_format` 인자를 이용하여, 데이터가 `user`, `item`, `rating`, `timestamp`의 4개의 컬럼으로 구성되어 있음을 알려준다.
- 로딩 시 `timestamp` 컬럼은 제외된다.

In [18]:
from surprise import Reader

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
data=Dataset.load_from_file('/content/ratings_noh.csv',reader=reader)

- 무비렌즈 데이터 형식이 아닌 다른 OS 파일의 경우에는 Reader 클래스를 먼저 설정해야 한다.
  - `line_format` : 컬럼을 순서대로 나욜, 입력된 문자열을 공백으로 분리하여 인식한다.
  - `sep(char)` : 컬럼을 분리하는 구분자. 디폴트는 `'\t'`.
  - `rating_scale(tuple, optional)` : 평점 값의 최소, 최대 평점을 설정한다.
- 이제 SVD 행렬 분해 기법을 이용해 추천을 예측해보자. 예측 평점과 실제 평점 데이터를 RMSE로 평가한다.

In [19]:
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

# 수행 시마다 동일한 결과 도출을 위해 random_state 설정 
algo = SVD(n_factors=50, random_state=0)

# 학습 데이터 세트로 학습 후 테스트 데이터 세트로 평점 예측 후 RMSE 평가
algo.fit(trainset) 
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8583


0.8582979491759881

##### **판다스 DataFrame에서 Surprise 데이터 세트로 로딩**
- `Dataset.load_from_df()`를 이용하면, 판다스의 DataFrame에서도 Surprise 데이터셋으로 로딩할 수 있다.
- 주의할 점은 DataFrame 역시 사용자 아이디, 아이템 아이디, 평점 컬럼 순서를 지켜야 한다.

In [20]:
import pandas as pd
from surprise import Reader, Dataset

ratings = pd.read_csv('/content/drive/MyDrive/2022-2 ESAA/Data/archive/ratings.csv') 
reader = Reader(rating_scale=(0.5, 5.0))

# ratings DataFrame 에서 컬럼은 사용자 아이디, 아이템 아이디, 평점 순서를 지켜야 함
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

trainset, testset = train_test_split(data, test_size=.25, random_state=0)

In [21]:
algo = SVD(n_factors=50, random_state=0)
algo.fit(trainset) 
predictions = algo.test( testset )
accuracy.rmse(predictions)

RMSE: 0.8583


0.8582979491759881

### | **Surprise 추천 알고리즘 클래스**
- Surprise에서 추천 예측을 위해 자주 사용되는 추천 알고리즘은 다음과 같다.
  + SVD : 행렬 분해를 통한 잠재 요인 협업 필터링을 위한 SVD 알고리즘
  + KNNBasic : 최근접 이웃 협업 필터링을 위한 KNN 알고리즘
  + BselineOnly : 사용자 Bias와 아이템 Bias를 감안한 SGD 베이스라인 알고리즘
- Surpirse SVD의 비용 함수는 사용자 베이스라인(Baseline) 편향성을 감안한 평점 예측에 Regularization을 적용한 것이다.
- SVD 클래스의 입력 파라미터는 다음과 같다. 
  - `n_factors` : 잠재 요인 K의 개수. 값이 커질수록 정확도가 높아지나, 과적합이 발생할 수 있다.
  - `n_epochs` : SGD 수행 시 반복 횟수
  - `biased(bool)` : 베이스라인 사용자 편향 적용 여부
- Baseline이란 **'각 개인이 평점을 부여하는 성향을 반영해 평점을 계산하는 방식'**이다.

### | **베이스라인 평점**
- 영화나 상품의 평가는 각 개인의 성향에 따라 같은 아이템이더라도 평가가 달라질 수 있다.
- 이러한 개인의 성향을 반영해 아이템 평가에 편향성(bias) 요소를 반영해 평점을 부과하는 것을 베이스라인 평점(Baseline Rating)이라고 한다.
- 베이스라인 평점 = 전체 평균 평점 + 사용자 편향 점수 + 아이템 편향 점수
  - 전체 평균 평점 : 모든 사용자의 아이템에 대한 평점의 평균
  - 사용자 편향 점수 = 사용자별 아이템 평점 평균 값 - 전체 평균 평점
  - 아이템 편향 점수 = 아이템별 평점 평균 값 - 전체 평균 평점

### | **교차 검증과 하이퍼 파라미터 튜닝**
- Surprise는 교차 검증과 하이퍼 파라미터 튜닝을 위해 사이킷런과 유사한 `cross_validate()`와 GridSearchCV 클래스를 제공한다.
- 먼저 교차 검증을 위한 `cross_validate()` 함수의 사용법부터 살펴보자. 해당 함수는 `surprise.model_selection` 모듈 내에 존재하며, 폴드된 데이터셋의 개수와 성능 측정 방법을 명시해 교차 검증을 수행한다.
- 다음 에제에서는 데이터를 5개의 학습/검증 폴드 데이터셋으로 나눠 교차 검증을 수행하고, RMSE, MAE로 성능 평가를 진행한다.

In [22]:
# 데이터 로딩 
ratings = pd.read_csv('/content/drive/MyDrive/2022-2 ESAA/Data/archive/ratings.csv') 
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

In [23]:
from surprise.model_selection import cross_validate 

algo = SVD(random_state=0) 
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True) 

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8718  0.8704  0.8657  0.8694  0.8705  0.8695  0.0021  
MAE (testset)     0.6712  0.6699  0.6676  0.6692  0.6706  0.6697  0.0013  
Fit time          4.74    5.89    6.00    5.61    6.54    5.76    0.59    
Test time         0.44    0.14    0.23    0.16    0.15    0.23    0.11    


{'test_rmse': array([0.87178814, 0.87035087, 0.86567531, 0.86937748, 0.87050342]),
 'test_mae': array([0.67122943, 0.66994339, 0.66759002, 0.66924324, 0.67061486]),
 'fit_time': (4.73964786529541,
  5.8906285762786865,
  6.001903533935547,
  5.612962007522583,
  6.540307521820068),
 'test_time': (0.44239211082458496,
  0.14489507675170898,
  0.23472929000854492,
  0.15702176094055176,
  0.15285539627075195)}

- `cross_validate()`는 위와 같이 폴드별 성능 평가 수치와 전체 폴드의 평균 성능 평가 수치를 함께 보여준다.
- Surprise의 GridSearchCV도 사이킷런의 GridSearchCV와 유사하게 교차 검증을 통해 하이퍼 파라미터 최적화를 수행한다.

In [24]:
from surprise.model_selection import GridSearchCV

# 최적화할 파라미터들을 딕셔너리 형태로 지정
param_grid = {'n_epochs': [20, 40, 60], 'n_factors': [50, 100, 200] }

# CV를 3개 폴드 세트로 지정, 성능 평가는 rmse, mse 로 수행 하도록 GridSearchCV 구성
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)

# 최고 RMSE Evaluation 점수와 그때의 하이퍼 파라미터
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.8712265103253783
{'n_epochs': 20, 'n_factors': 50}


### | **Surprise를 이용한 개인화 영화 추천 시스템 구축**
- Surprise를 이용해 잠재 요인 협업 필터링 기반의 개인화된 영화 추천을 구현해보자.
- Surprise 패키지로 학습된 추천 알고리즘을 기반으로 특정 사용자가 아직 평점을 매기지 않은 영화 중 개인 취향에 가장 적절한 영화를 추천한다.
- `ratings.csv` 데이터를 학습/테스트 데이터로 분리하지 않고 전체를 학습 데이터셋으로 사용한다.
- 그러나 Surprise는 `train_test_split()`을 이용해 내부에서 사용하는 TrainSet 클래스로 변환하지 않으면 `fit()`을 이용해 학습할 수 없다.

In [25]:
# 아래 코드는 train_test_split()으로 분리되지 않는 Dataset에 fit()을 호출하여 오류를 발생
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
algo = SVD(n_factors=50, random_state=0)
algo.fit(data)

AttributeError: ignored

- 따라서 데이터셋 전체를 학습 데이터로 사용하기 위해 DatasetAutoFolds 클래스를 이용한다.

In [26]:
from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))

# DatasetAutoFolds 클래스를 ratings_noh.csv 파일 기반으로 생성. 
data_folds = DatasetAutoFolds(ratings_file='/content/ratings_noh.csv', reader=reader)

In [27]:
# 전체 데이터를 학습데이터로 생성
trainset = data_folds.build_full_trainset()

- 이제 이렇게 생성된 학습 데이터를 기반으로 학습을 수행해보자.

In [28]:
algo = SVD(n_epochs=20, n_factors=50, random_state=0)
algo.fit(trainset)

- 특정 사용자를 `userId=9`인 사용자로 지정하여, 이 사용자가 아직 보지 않은 영화 목록을 확인해보자.

In [29]:
# 영화에 대한 상세 속성 정보 DataFrame로딩
movies = pd.read_csv('/content/drive/MyDrive/2022-2 ESAA/Data/archive/movies.csv')

# userId=9 의 movieId 데이터 추출하여 movieId=42 데이터가 있는지 확인
movieIds = ratings[ratings['userId']==9]['movieId']

if movieIds[movieIds==42].count() == 0:
    print('사용자 아이디 9는 영화 아이디 42의 평점 없음')

print(movies[movies['movieId']==42])

사용자 아이디 9는 영화 아이디 42의 평점 없음
    movieId                   title              genres
39       42  Dead Presidents (1995)  Action|Crime|Drama


- 이 `movieId`가 42인 영화에 대한 사용자의 추천 예상 평점을 `predict()` 메서드를 이용하여 계산해보자.

In [30]:
uid = str(9) # 문자열
iid = str(42) # 문자열

pred = algo.predict(uid, iid, verbose=True)

user: 9          item: 42         r_ui = None   est = 2.67   {'was_impossible': False}


- 추천 예측 평점인 `est` 값은 2.67이다.
- 이제 사용자가 평점을 매기지 않은 전체 영화를 추출한 뒤, 예측 평점 순으로 영화를 추천해보자.

In [31]:
def get_unseen_surprise(ratings, movies, userId):
    #입력값으로 들어온 userId에 해당하는 사용자가 평점을 매긴 모든 영화를 리스트로 생성
    seen_movies = ratings[ratings['userId']== userId]['movieId'].tolist()
    
    # 모든 영화들의 movieId를 리스트로 생성. 
    total_movies = movies['movieId'].tolist()
    
    # 모든 영화들의 movieId중 이미 평점을 매긴 영화의 movieId를 제외하여 리스트로 생성
    unseen_movies= [movie for movie in total_movies if movie not in seen_movies]
    print('평점 매긴 영화수:',len(seen_movies), '추천대상 영화수:',len(unseen_movies), \
          '전체 영화수:',len(total_movies))
    
    return unseen_movies

unseen_movies = get_unseen_surprise(ratings, movies, 9)

평점 매긴 영화수: 121 추천대상 영화수: 10208 전체 영화수: 10329


- 사용자 아이디 9번은 전체 데이터 10,329개의 데이터 중 10,208개만 평점을 매겼다.
- 이 중 앞에서 학습된 추천 알고리즘 클래스인 SVD를 이용해 높은 평점을 가진 순으로 영화를 추천해보자.

In [32]:
def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n=10):
    # 알고리즘 객체의 predict() 메서드를 평점이 없는 영화에 반복 수행한 후 결과를 list 객체로 저장
    predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]
    
    # predictions list 객체는 surprise의 Predictions 객체를 원소로 가지고 있음.
    # [Prediction(uid='9', iid='1', est=3.69), Prediction(uid='9', iid='2', est=2.98),,,,]
    # 이를 est 값으로 정렬하기 위해서 아래의 sortkey_est 함수를 정의함.
    # sortkey_est 함수는 list 객체의 sort() 함수의 키 값으로 사용되어 정렬 수행.
    def sortkey_est(pred):
        return pred.est
    
    # sortkey_est( ) 반환값의 내림 차순으로 정렬 수행하고 top_n개의 최상위 값 추출.
    predictions.sort(key=sortkey_est, reverse=True)
    top_predictions= predictions[:top_n]
    
    # top_n으로 추출된 영화의 정보 추출. 영화 아이디, 추천 예상 평점, 제목 추출
    top_movie_ids = [ int(pred.iid) for pred in top_predictions]
    top_movie_rating = [ pred.est for pred in top_predictions]
    top_movie_titles = movies[movies.movieId.isin(top_movie_ids)]['title']
    top_movie_preds = [ (id, title, rating) for id, title, rating in zip(top_movie_ids, top_movie_titles, top_movie_rating)]
    
    return top_movie_preds

unseen_movies = get_unseen_surprise(ratings, movies, 9)
top_movie_preds = recomm_movie_by_surprise(algo, 9, unseen_movies, top_n=10)
print('##### Top-10 추천 영화 리스트 #####')

for top_movie in top_movie_preds:
    print(top_movie[1], ":", top_movie[2])

평점 매긴 영화수: 121 추천대상 영화수: 10208 전체 영화수: 10329
##### Top-10 추천 영화 리스트 #####
Citizen Kane (1941) : 3.6567993015028364
Monty Python and the Holy Grail (1975) : 3.6441579884085935
Ran (1985) : 3.60144346888342
Graduate, The (1967) : 3.5649809722895442
Touch of Evil (1958) : 3.5444497313393666
Princess Mononoke (Mononoke-hime) (1997) : 3.5424025472565672
Requiem for a Dream (2000) : 3.536964525291929
Amores Perros (Love's a Bitch) (2000) : 3.520333145002889
Nausicaä of the Valley of the Wind (Kaze no tani no Naushika) (1984) : 3.518017486366034
Intouchables (2011) : 3.5115619895086674
